In [1]:
from PIL import Image
from rl_benchmarks.models import iBOTViT
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator
import pathlib
from tqdm import tqdm
from PIL import Image
import numpy as np
Image.MAX_IMAGE_PIXELS = None
from torchvision import transforms
import torch
from torch.utils.data import Dataset
import os
from multiprocessing import Pool
import umap
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from rl_benchmarks.utils.linear_evaluation import get_binary_class_metrics, get_bootstrapped_metrics

from PIL import Image
import pathlib
from tqdm import tqdm
from PIL import Image
import numpy as np
Image.MAX_IMAGE_PIXELS = None
from torchvision import transforms
import torch
from torch.utils.data import Dataset
import os
from multiprocessing import Pool
# import umap
import numpy as np
import matplotlib.pyplot as plt
import sklearn

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

from sklearn.metrics import f1_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, cohen_kappa_score, accuracy_score
import torch.nn.functional as F
import sys
import time

import shutil
import os

# import albumentations as A
# from albumentations.pytorch import ToTensorV2
import cv2
import time
from sklearn.preprocessing import label_binarize

import timm
from metrics import report
from pathlib import Path

import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset, Dataset
from tqdm import tqdm
from rl_benchmarks.metrics import *

from rl_benchmarks.trainers.torch_trainer import TorchTrainer
from rl_benchmarks.models.slide_models.meanpool import MeanPool
from rl_benchmarks.models.slide_models.chowder import Chowder
from rl_benchmarks.models.slide_models.dsmil import DSMIL
from rl_benchmarks.models.slide_models.abmil import ABMIL
from rl_benchmarks.models.slide_models.hiptmil import HIPTMIL
from rl_benchmarks.models.slide_models.transmil import TransMIL

from pathlib import Path
from metrics import report

import sys
sys.path.append('/home/yuxin/bme/BCaCAD/model')
from patch_based_test.img import QiLuROI


In [2]:
class IBOTMultiTaskModel(nn.Module):
    def __init__(self, num_classes):
        super(IBOTMultiTaskModel, self).__init__()
        weights_path = '/home/yuxin/Downloads/ibot_vit_base_pancan.pth'
        self.base_model = iBOTViT(architecture="vit_base_pancan", encoder="teacher", weights_path=weights_path)
        # print(self.base_model)
        self.num_features = 768
        self.num_classes = num_classes

        if isinstance(num_classes, list):
            self.heads = nn.ModuleList([nn.Linear(self.num_features, num_class) for num_class in num_classes])
        else:
            self.head = self.base_model.head
            self.head.fc = nn.Linear(self.num_features, num_classes)

    def forward(self, x):
        # Forward pass through the base model
        x = self.base_model(x)
        if isinstance(self.num_classes, list):
            x = [head(x) for head in self.heads]
        else:
            x = self.head(x)
        return x

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
num_classes = [3,3]
img_size = patch_size = 384
data_trans = {
    "train": transforms.Compose([
                                transforms.Resize(img_size),
                                transforms.ColorJitter(),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    "test": transforms.Compose([
                                transforms.Resize((img_size,img_size)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

In [4]:
feature_model = IBOTMultiTaskModel(num_classes)
device = "cuda"
feature_model.to(device)
feature_model = nn.DataParallel(feature_model)
weight_path = '/mnt/hd0/project/bcacad/model/pretrainSSL_ibot_vit+ibot_ft+fsl_ft/model-5.pth'
feature_model.load_state_dict(torch.load(weight_path, map_location=device)['model_state_dict'])
model = feature_model


2024-03-19 09:44:18.190 | INFO     | rl_benchmarks.models.feature_extractors.ibot_vit:__init__:78 - Pretrained weights found at /home/yuxin/Downloads/ibot_vit_base_pancan.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v', 'head.last_layer2.weight_g', 'head.last_layer2.weight_v'])


In [18]:
class PatchResultFly(Dataset):
    def get_label(self, type):
        D = {
            'normal':[0,0],
            'dcis-1':[1,0],
            'dcis-2':[1,1],
            'dcis-3':[1,2],
            'ibc-1':[2,0],
            'ibc-2':[2,1],
            'ibc-3':[2,2],
        }
        label =  D[type]
        return label[0],label[1]
    
    def __init__(self, image_dir, phase='test', train_tile=None):
        folder = Path(image_dir)
        self.im_paths = list(folder.rglob('**/*.*'))
        self.labels = [self.get_label(path.parent.name) for path in self.im_paths]
        self.phase=phase
        self.train_tile = train_tile
        self.size = 336
        self.bs = 16
        self.src_mag = 10
        self.tar_mag = 10
    
        
    def __len__(self):
        return len(self.im_paths)
    def __getitem__(self, item):
        im_path = self.im_paths[item]
        im = QiLuROI(str(im_path), self.src_mag, self.tar_mag, self.size)
        im.setIterator(self.size)
        patches = [p for p in im]
        if self.phase == 'train':
            indices = np.random.choice(len(patches), self.train_tile, replace=True)
            patches = [patches[i] for i in indices]
        patches = [data_trans[self.phase](p) for p in patches]
        bs = self.bs
        for i in range(0, len(patches), bs):
            x = torch.stack(patches[i:i+bs], dim=0)
            x = x.to(device)
            y = model(x)
            y = torch.concatenate([torch.sigmoid(y[0]).unsqueeze(1),torch.sigmoid(y[1]).unsqueeze(1)], dim=1)
            y = y.detach().cpu().numpy()
            if i == 0:
                output = y
            else:
                # output = torch.concatenate([output, y], dim=0)
                np.concatenate([output, y], axis=0)
        return output, [self.labels[item]]*len(patches)



In [33]:
data_root = Path('/mnt/hd0/project/bcacad/data/roi-level')
test_cohorts = ['suqh_full', 'qduh', 'shsu', 'bracs', 'bcnb', 'bach', 'apght']

In [7]:
tasks = ['type', 'nonibc', 'ibc']
class_names = {
    'type': ['Normal', 'nonIBC', 'IBC'],
    'nonibc': ['Low', 'Medium', 'High'],
    'ibc': ['Low', 'Medium', 'High'],
}

In [39]:
# testing
pha='test'
# test_cohorts = [ 'qduh']
test_cohorts = ['bcnb', 'bach', 'apght']
for cohort in test_cohorts:
    test_set = PatchResultFly(data_root / cohort / 'test', 'test')
    probs = [[],[]]
    labels = [[],[]]
    preds = [[],[]]
    for i in range(len(test_set)):
        output, label = test_set[i]
        for j in range(len(output)):
            for l in range(2):
                pred = np.argmax(output[j][l])
                probs[l].append(output[j][l].tolist())
                labels[l].append(label[j][l])
                preds[l].append(pred)

    labels = np.array(labels)
    probs = np.array(probs)
    preds = np.array(preds)

    type_labels = labels[0]
    type_probs = probs[0]
    type_preds = preds[0]

    nonibc_index = np.where(type_labels ==1)
    nonibc_labels = labels[1][nonibc_index]
    nonibc_probs = probs[1][nonibc_index]
    nonibc_preds = preds[1][nonibc_index]

    ibc_index = np.where(type_labels ==2)
    ibc_labels = labels[1][ibc_index]
    ibc_probs = probs[1][ibc_index]
    ibc_preds = preds[1][ibc_index]

    re = {}
    avg_aucs = {}
    if type_labels.size == 0:
        re['type'] = None
        avg_aucs['type'] = None
    else:
        re['type'] = report(type_labels, type_preds, type_probs, class_names['type'])
        avg_aucs['type'] = compute_mean_one_vs_all_auc(type_labels, type_probs)

    if nonibc_labels.size == 0:
        re['nonibc'] = None
        avg_aucs['nonibc'] = None
    else:
        re['nonibc'] = report(nonibc_labels, nonibc_preds, nonibc_probs, class_names['nonibc'])
        avg_aucs['nonibc'] = compute_mean_one_vs_all_auc(nonibc_labels, nonibc_probs)

    if ibc_labels.size == 0:
        re['ibc'] = None
        avg_aucs['ibc'] = None
    else:
        re['ibc'] = report(ibc_labels, ibc_preds, ibc_probs, class_names['ibc'])
        avg_aucs['ibc'] = compute_mean_one_vs_all_auc(ibc_labels, ibc_probs)

    for task in ['type', 'nonibc', 'ibc']:
        r = re[task]
        if r is None:
            continue
        fs = "{} {} acc: {:.4f}, auc: {:.4f} [{:.4f} {:.4f} {:.4f}]".format(cohort, task, r['accuracy'], avg_aucs[task], r['0']['auc'], r['1']['auc'], r['2']['auc'])
        print(fs)
    print()

bcnb type acc: 0.7858, auc: nan [nan nan nan]
bcnb ibc acc: 0.4883, auc: 0.6712 [0.7749 0.5410 0.6975]

bach type acc: 0.9333, auc: 0.9880 [0.9938 0.9810 0.9890]
bach nonibc acc: 0.1025, auc: nan [nan nan nan]
bach ibc acc: 0.0925, auc: nan [nan nan nan]

apght type acc: 0.9033, auc: nan [nan nan nan]
apght ibc acc: 0.3867, auc: 0.6483 [0.8043 0.4640 0.6769]

